In [2]:
 def entroper(p):   
    import math
    entropy = 0
    for i in p:
        if i == 0:
            entropy += 0
        else:
            entropy += i*math.log(i,2)
    return abs(entropy)

In [3]:
def modify(listt):
    string = listt[0]
    stringlist = string.split("[ ")
    stringlist.pop(0)
    modified = stringlist[0]
    listt.pop(0)
    listt.insert(0,modified)

In [4]:
# Reading the files
def motiffetch(location):
    import pandas as pd

    pssm = open(location,"r")
    lines = pssm.readlines()
    motifdict = {}
    motif = 0
    motiflist = []
    for line in lines:
        if line[0] == "A":
            motiflist = []
            motif += 1
            linelist = line.split("\t")
            linelist.remove("A")
            linelist.remove("]\n")
            modify(linelist)
            motiflist.append(linelist)
        if line[0] == "G" or line[0] == "C":
            linelist = line.split("\t")
            if "G" in linelist:
                linelist.remove("G")
            if "C" in linelist:
                linelist.remove("C")
            linelist.remove("]\n")
            modify(linelist)
            motiflist.append(linelist)
        if line[0] == "T":
            linelist = line.split("\t")
            motiflist.append(linelist)
            linelist.remove("T")
            linelist.remove("]\n")
            modify(linelist)
            motifdict[motif] = motiflist
    return motifdict

In [5]:
def arraymaker(loc):
    import pandas as pd
    dic = motiffetch(loc)
    for i in dic.keys():
        df = pd.DataFrame(dic[i])
        ls = []
        for j in df.columns:
            total = (df[j].astype(float)).sum()
            df[j] = (df[j].astype(float))/total
            ls.append(df[j].tolist())
        dic[i] = ls
    return dic

In [6]:
def edic(loc):
    c = arraymaker(loc)
    import pandas as pd
    dic = motiffetch(loc)
    entropydic = {}
    for i in dic.keys():
        df = pd.DataFrame(dic[i])
        for j in df.columns:
            total = (df[j].astype(float)).sum()
            df[j] = (df[j].astype(float))/total
        entropylist = []
        for k in df.columns:
            column = df[k].tolist()
            entropylist.append(entroper(column))
        entropydic[i] = entropylist
    return [c,entropydic]

In [7]:
def revdic(locs):
    diclist = edic(locs)
    dic = diclist[0]
    oldedic = diclist[1]
    newdic = {}
    for i in dic.keys():
        pos = dic[i]
        revpos = pos[::-1]
        newpos = []
        for j in range(len(revpos)):
            oldprob = revpos[j]
            newprob = oldprob[::-1]
            newpos.append(newprob)
        newdic[i] = newpos
    newedic = {}
    for k in oldedic.keys():
        olde = oldedic[k]
        newe = olde[::-1]
        newedic[k] = newe
    return [newdic,newedic]
        

In [8]:
def generator(dic,index):
    import random
    values = dic[index]
    width = len(values[0])
    characters = ["A","C","G","T"]
    letters = []
    for i in range(width):
        weights = []
        for j in range(4):
            weight = float(values[j][i])
            weights.append(weight)
        element = random.choices(characters,weights,k=1)
        letter = "".join(element)
        letters.append(letter)
    word = "".join(letters)
    return word

In [9]:
def judge(m1,m2,e1,e2):
    motif1 = generator(m1[0],m1[1])
    motif2 = generator(m2[0],m2[1])
    if motif1 > motif2:
        big = motif1
        small = motif2
        bige = e1[m1[1]]
        smalle = e2[m2[1]]
    else:
        small = motif1
        big = motif2
        bige = e2[m2[1]]
        smalle = e1[m1[1]]
    checkindex = len(big) - len(small)
    result = 0
    for i in range(checkindex+1):
        equal = big[i:i+len(small)]
        equale = bige[i: i + len(small)]
        w1 = list(small)
        w2 = list(equal)
        matchcount = 0
        for j in range(len(w1)):
            if w1[j] == w2[j]:
                if smalle[j] < 1 and equale[j] < 1:
                    matchcount +=1
        if matchcount > result :
            result = matchcount
    ecount = 0
    for k in range(len(smalle)):
        if smalle[k] < 1:
            ecount +=1
    if result >= (ecount-1):
        return True
            

In [10]:
def jscore(m1,m2,e1,e2):
    from scipy.spatial import distance
    if len(m1) > len(m2):
        big = m1
        small = m2
        bige = e1
        smalle = e2
    else:
        small = m1
        big = m2
        bige = e2
        smalle = e1
    checkindex = len(big) - len(small)
    score = 0
    for i in range(checkindex+1):
        equal = big[i:i+len(small)]
        equale = bige[i: i + len(small)]
        count = 0
        for j in range(len(equal)):
            js = distance.jensenshannon(equal[j],small[j])
            if js < 0.6 and equale[j] < 1 and smalle[j] < 1:
                count += 1
        if count > score:
            score = count
    return score              

In [11]:
def weed(m,e):
    ecount = 0
    mcount = 0
    for i in range(0,len(e)):
        if e[i] < 1 :
            ecount +=1
    if ecount < 10:
        return False
    else:
        return True
    

In [12]:
loc1 = "../Files/Data/pssms2.txt"
loc2 = "../Files/Data/pssms8.txt"
k1 = edic(loc1)
k2 = edic(loc2)
k2r = revdic(loc2)
e1 = k1[1]
e2 = k2[1]
print(weed(e1,e2[1]))
e2r = k2r[1]
ms1 = k1[0]
ms2 = k2[0]
ms2r = k2r[0]
for i in ms1.keys():
    for j in ms2.keys():
        if weed(ms1[i],e1[i]) is False and weed(ms2[j],e2[j]) is False:
            if jscore(ms1[i],ms2[j],e1[i],e2[j]) > 3:
                print(jscore(ms1[i],ms2[j],e1[i],e2[j]),i,j)
        if weed(ms1[i],e1[i]) is False and weed(ms2r[j],e2r[j]) is False:
            if jscore(ms1[i],ms2r[j],e1[i],e2r[j]) > 3:
                print(jscore(ms1[i],ms2r[j],e1[i],e2r[j]),i,j,"rev")

False
5 1 2
4 1 3
4 1 4
4 1 5
5 1 5 rev
5 1 6 rev
5 2 1
4 5 2 rev
4 5 5
5 5 5 rev
4 5 6 rev


In [13]:
def matchfinder():
    import pandas as pd
    loclist = [1,2,3,4,5,6,7,8,9,10,11,12]
    matchdic = {}
    for i in loclist:
        matchlist = []
        for j in loclist:
            if i != j:
                loc1 = "../Files/Data/pssms" + str(i) + ".txt"
                loc2 = "../Files/Data/pssms" + str(j) + ".txt"
                k1 = edic(loc1)
                k2 = edic(loc2)
                k2r = revdic(loc2)
                e1 = k1[1]
                e2 = k2[1]
                e2r = k2r[1]
                ms1 = k1[0]
                ms2 = k2[0]
                ms2r = k2r[0]
                paired = 0
                for k in ms1.keys():
                    partner = 0
                    partnerlength = 0
                    for l in ms2.keys():
                        if weed(ms1[k],e1[k]) is False and weed(ms2[l],e2[l]) is False:
                            if jscore(ms1[k],ms2[l],e1[k],e2[l]) > 4:
                                if partner == 0 :
                                    partner = l
                                    partnerlength = len(e2)
                                else:
                                    if len(e2) < partnerlength:
                                        partner = l
                                        partnerlength = len(e2)
                                    else:
                                        continue
                            elif jscore(ms1[k],ms2r[l],e1[k],e2r[l]) > 4:
                                if jscore(ms1[k],ms2r[l],e1[k],e2r[l]) > jscore(ms1[k],ms2[l],e1[k],e2[l]):
                                    if partner == 0 :
                                        partner = l
                                        partnerlength = len(e2r[l])
                                    else:
                                        if len(e2[l]) < partnerlength:
                                            partner = l
                                            partnerlength = len(e2r[l])
                                        else:
                                            continue
                    if partner != 0:
                            paired +=1
            if i == j:
                paired = -1
            matchlist.append(paired)
        matchdic[i] = matchlist
    df = pd.DataFrame(matchdic)
    df.style.apply(lambda x:"background-color : red" if x > 3 else '')
    return df

matchfinder()

,1,2,3,4,5,6,7,8,9,10,11,12
0,-1,3,1,2,2,0,2,2,2,1,1,3
1,2,-1,2,2,3,1,4,4,2,2,1,9
2,1,2,-1,2,1,1,1,2,1,0,0,1
3,2,3,2,-1,4,2,4,4,3,1,1,5
4,2,3,1,3,-1,2,3,4,4,0,1,6
5,0,1,1,2,2,-1,1,1,2,0,0,0
6,2,4,1,3,4,1,-1,5,3,1,0,7
7,2,3,2,3,4,1,4,-1,3,0,0,4
8,2,3,1,3,5,2,4,4,-1,1,1,6
9,1,2,0,1,0,0,1,0,1,-1,1,6


In [15]:
def highlighter(x):
    condition = x[0] > 3 
    bg_color = 'background-color : #242020' if condition else ''
    return bg_color

def weedf():
    import numpy as np
    df0 = matchfinder()
    df = df0.iloc[:,np.arange(1,len(df0.columns))]
    styled = df.style.applymap(highlighter)
    return sty